In [1]:
import re
import math
import time

def load_data(is_test=False):
    if is_test:
        input_txt = "input_.txt"
    else:
        input_txt = "input.txt"
        
    with open(input_txt) as f:
        lines = f.read().strip().split('\n\n')

    seeds                = line2numbers(lines[0].replace('seeds: ', ''))
    seed2soil            = extract_mappings(lines[1], 'seed', 'soil')
    soil2fertilizer      = extract_mappings(lines[2], 'soil', 'fertilizer')
    fertilizer2water     = extract_mappings(lines[3], 'fertilizer', 'water')
    water2light          = extract_mappings(lines[4], 'water', 'light')
    light2temperature    = extract_mappings(lines[5], 'light', 'temperature')
    temperature2humidity = extract_mappings(lines[6], 'temperature', 'humidity')
    humidity2location    = extract_mappings(lines[7], 'humidity', 'location')

    return (seeds, seed2soil, soil2fertilizer, fertilizer2water, water2light, light2temperature, temperature2humidity, humidity2location)

In [2]:
def line2numbers(line):
    return [int(x) for x in re.findall('\d+', line)]


def extract_mappings(line, source, destination):
    line_ = line.replace(source + '-to-' + destination + ' map:\n', '')
    return [line2numbers(x) for x in line_.split('\n')]

## part 1

In [3]:
def _convert(source, mapping):
    destination_begin, source_begin, range_length = mapping
    destination = source
    if (source >= source_begin) and (source < source_begin+range_length):
        destination += destination_begin - source_begin
    return destination


def convert(source, mappings):
    destination = source 
    for mapping in mappings: 
        #destination = _convert(destination, mapping)
        destination = _convert(source, mapping)
        if destination != source:
            break
    return destination


def seed2location(seed, data, debug=False):
    # this function can be written shortly using loop,
    # but for the debug purpose
    # just wrote down all the names.
    (seeds, 
     seed2soil, 
     soil2fertilizer, 
     fertilizer2water, 
     water2light, 
     light2temperature, 
     temperature2humidity, 
     humidity2location) = data
    
    soil        = convert(seed, seed2soil)
    fertilizer  = convert(soil, soil2fertilizer)
    water       = convert(fertilizer, fertilizer2water)
    light       = convert(water, water2light)
    temperature = convert(light, light2temperature)
    humidity    = convert(temperature, temperature2humidity)
    location    = convert(humidity, humidity2location)

    if debug:
        print(f"soil: {soil}")
        print(f"fertilizer: {fertilizer}")
        print(f"water: {water}")
        print(f"light: {light}")
        print(f"temperature: {temperature}")
        print(f"humidity: {humidity}")
        print(f"location: {location}")

    return location

In [4]:
data = load_data(is_test=False)
(seeds, seed2soil, soil2fertilizer, fertilizer2water, water2light, light2temperature, temperature2humidity, humidity2location) = data
time_start = time.time()
locations = [seed2location(seed, data) for seed in seeds]
print(f"elapsed time: {time.time() - time_start:.6f} [sec]")

#print(f"seeds: {seeds}")
#print(f"locations: {locations}")
print(f"the lowest location number that corresponds to any of the initial seed numbers: {min(locations)}")

elapsed time: 0.000407 [sec]
the lowest location number that corresponds to any of the initial seed numbers: 51752125


## part 2
if seed2location() is executed to all seeds, it will take more than 200 hours.  
therefore we will search possible seeds from locations.

In [5]:
## check estimated time.
data = load_data(is_test=False)
(seeds, seed2soil, soil2fertilizer, fertilizer2water, water2light, light2temperature, temperature2humidity, humidity2location) = data
#print(f"seeds: {seeds}")

total = 0
for i in range(0, len(seeds), 2):
    total += seeds[i+1]
print(f"number of options: {total}")

estimated_time = 0.0016 / 4 * total / 3600
print(f"estimated_time: {estimated_time:.2f}[hours]")

number of options: 2104769314
estimated_time: 233.86[hours]


In [6]:
## inverse conversion.
def _convert_inverse(destination, mapping):
    destination_begin, source_begin, range_length = mapping
    source = destination
    if (destination >= destination_begin) and (destination < destination_begin+range_length):
        source += source_begin - destination_begin
    return source


def convert_inverse(destination, mappings):
    source = destination
    for mapping in mappings: 
        source = _convert_inverse(destination, mapping)
        if source != destination:
            break
    return source


def location2seed(location, data, debug=False):
    # this function can be written shortly using loop,
    # but for the debug purpose
    # just wrote down all the names.
    (seeds, 
     seed2soil, 
     soil2fertilizer, 
     fertilizer2water, 
     water2light, 
     light2temperature, 
     temperature2humidity, 
     humidity2location) = data
    
    humidity    = convert_inverse(location, humidity2location)
    temperature = convert_inverse(humidity, temperature2humidity)
    light       = convert_inverse(temperature, light2temperature)
    water       = convert_inverse(light, water2light)
    fertilizer  = convert_inverse(water, fertilizer2water)
    soil        = convert_inverse(fertilizer, soil2fertilizer)
    seed        = convert_inverse(soil, seed2soil)    

    if debug:
        print(f"location: {location}")
        print(f"humidity: {humidity}")
        print(f"temperature: {temperature}")
        print(f"light: {light}")
        print(f"water: {water}")
        print(f"fertilizer: {fertilizer}")        
        print(f"soil: {soil}")

    return seed


def seed_in_range(x, seeds):
    for i in range(0, len(seeds), 2):
        result = False
        #print(f"{seeds[i]}--{seeds[i]+seeds[i+1]}")
        if (x >= seeds[i]) and (x < seeds[i]+seeds[i+1]):
            result = True
            break
    return result

In [7]:
## check if the inverse function works.
# locations = [seed2location(seed, data) for seed in seeds]
# print(locations)
# print([location2seed(location, data) for location in locations])
# print(seeds)

In [9]:
location_min = seed2location(seeds[0], data)

# check if the seed_ derived from location_ is in the range.
i = 0
location_ = seed2location(seeds[0], data) # next location_min option
print(f"start location: {location_}")

#while(location__ != location_):
time_start = time.time()
while(i < 50000):
    location__ = location_
    seed_ = location2seed(location_, data)
    if seed_in_range(seed_, seeds):
        location_min = location_
        location_ = math.ceil(location_min/2)
    else:
        location_ = math.ceil((location_min + location_)/2)
        #print(f"\r[{i}] location_: {location_}____", end="")
    #print(f"[{i}] location_min: {location_min} | location_: {location_}")
    i += 1
print(f"elapsed time: {time.time() - time_start:.3f}[sec]")
print(f"the lowest location number that corresponds to any of the initial seed numbers: {location_min}")

start location: 3426327245
elapsed time: 0.401[sec]
the lowest location number that corresponds to any of the initial seed numbers: 12634632


In [10]:
# check if the location really exists.
seed = location2seed(location_min, data)
print(seed)
seed_in_range(seed, seeds)

692213654


True